In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import plotly.express as px

In [ ]:
from utils_s2_tree_search import StemBbTree, BoundingBox, bb_conflict

In [ ]:
df = pd.read_pickle('data/data_len40_1000_s1_stem_bb.pkl.gz')


In [ ]:
# # subset to example with bb sensitivity = 100%
# df = df[df['bb_identical'] == 1]

data = []

for _, row in df.iterrows():

    df_stem = pd.DataFrame(row.pred_stem_bb)

    # we use df index, make sure it's contiguous
    assert df_stem.iloc[-1].name == len(df_stem) - 1

    bbs = {}
    for idx, r in df_stem.iterrows():
        bbs[idx] = BoundingBox(bb_x=r['bb_x'],
                               bb_y=r['bb_y'],
                               siz_x=r['siz_x'],
                               siz_y=r['siz_y'])

    bb_conf_arr = np.zeros((len(bbs), len(bbs)))
    for i in bbs.keys():
        bb1 = bbs[i]
        for j in bbs.keys():
            bb2 = bbs[j]
            # TODO only need to compute half
            bb_conf_arr[i, j] = bb_conflict(bb1, bb2)
    assert np.all(bb_conf_arr.T == bb_conf_arr)
    
    bb_tree = StemBbTree(bbs, bb_conf_arr)
    
    data.append({
        'n_bbs': len(bbs),
        'n_valid_combos': len(bb_tree.leaves),
    })

In [ ]:
data = pd.DataFrame(data)

In [ ]:
px.scatter(data, x='n_bbs', y='n_valid_combos')

In [ ]:
len(df)